# Day 5: Supply Stacks
## Part 1

The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked crates, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a giant cargo crane capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her which crate will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates and the rearrangement procedure (your puzzle input). For example:

```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
```

In this example, there are three stacks of crates. Stack 1 contains two crates: crate Z is on the bottom, and crate N is on top. Stack 2 contains three crates; from bottom to top, they are crates M, C, and D. Finally, stack 3 contains a single crate, P.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:

```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
```
 
In the second step, three crates are moved from stack 1 to stack 3. Crates are moved one at a time, so the first crate to be moved (D) ends up below the second and third crates:

```
        [Z]
        [N]
    [C] [D]
    [M] [P]
 1   2   3
```

Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved one at a time, crate C ends up below crate M:

```
        [Z]
        [N]
[M]     [D]
[C]     [P]
 1   2   3
```

Finally, one crate is moved from stack 1 to stack 2:

```
        [Z]
        [N]
        [D]
[C] [M] [P]
 1   2   3
```

The Elves just need to know which crate will end up on top of each stack; in this example, the top crates are C in stack 1, M in stack 2, and Z in stack 3, so you should combine these together and give the Elves the message CMZ.

After the rearrangement procedure completes, what crate ends up on top of each stack?

In [1]:
import re
import copy
import numpy as np

In [2]:
def parse_start_state(stack_diagram: str):
    """Take the text based representation of the initial stack state and convert it into a list of lists
    """
    start_state = np.array(re.findall(r'(?:(\[\w\]|\s{3})\s)', stack_diagram, re.MULTILINE)).reshape(8,9)
    start_state = np.char.strip(start_state)
    start_state = np.char.replace(start_state, '[', '')
    start_state = np.char.replace(start_state, ']', '')
    start_state = start_state.T.tolist()
    start_state = [ list(''.join(stack))[::-1] for stack in start_state ]
    return start_state

In [3]:
def move_crates(num: int, stack_from: int, stack_to: int, state: list):
    """Move crates on at a time from one stack to another using list pop and extend
    """
    for i in range(num):
        crate = state[stack_from].pop()
        state[stack_to].extend(crate)

In [4]:
def move_multiple_crates(num: int, stack_from: int, stack_to: int, state: list):
    """Move multiple crate at once. Store the crates popped off the stack and add them,
    preserving original ordering onto the other stack
    """
    temp_stack = []
    for i in range(num):
        temp_stack.extend(state[stack_from].pop())
    state[stack_to].extend(temp_stack[::-1])

In [5]:
# Split the puzzle input
stack_diagram, part1 = open("input.txt", "r").read().split('\n\n')
# Parse the initial stack state into a list of lists
start_state = parse_start_state(stack_diagram)
# Make a separate copy of the starting state to mutate during part1
part1_state = copy.deepcopy(start_state)
# Parse out all the moves into tuples
moves = re.findall(r'\w* (\d+) \w* (\d+) \w* (\d+)', part1, re.MULTILINE)
# Process each move by moving one crate at a time
for move in moves:
    move_crates(int(move[0]), int(move[1])-1, int(move[2])-1, part1_state)
# Print the top crate in each stack
''.join([ crate[-1] for crate in part1_state ])

'TGWSMRBPN'

## Part Two

As you watch the crane operator expertly rearrange the crates, you notice the process isn't following your prediction.

Some mud was covering the writing on the side of the crane, and you quickly wipe it away. The crane isn't a CrateMover 9000 - it's a CrateMover 9001.

The CrateMover 9001 is notable for many new and exciting features: air conditioning, leather seats, an extra cup holder, and the ability to pick up and move multiple crates at once.

Again considering the example above, the crates begin in the same configuration:

```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 
```

Moving a single crate from stack 2 to stack 1 behaves the same as before:

```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
```

However, the action of moving three crates from stack 1 to stack 3 means that those three moved crates stay in the same order, resulting in this new configuration:

```
        [D]
        [N]
    [C] [Z]
    [M] [P]
 1   2   3
```

Next, as both crates are moved from stack 2 to stack 1, they retain their order as well:

```
        [D]
        [N]
[C]     [Z]
[M]     [P]
 1   2   3
```

Finally, a single crate is still moved from stack 1 to stack 2, but now it's crate C that gets moved:

```
        [D]
        [N]
        [Z]
[M] [C] [P]
 1   2   3
```

In this example, the CrateMover 9001 has put the crates in a totally different order: MCD.

Before the rearrangement process finishes, update your simulation so that the Elves know where they should stand to be ready to unload the final supplies. After the rearrangement procedure completes, what crate ends up on top of each stack?

In [6]:
# Make a separate copy of the starting state to mutate during part2
part2_state = copy.deepcopy(start_state)
# Process each move by moving multiple crate at once
for move in moves:
    move_multiple_crates(int(move[0]), int(move[1])-1, int(move[2])-1, part2_state)
# Print the top crate in each stack
''.join([ crate[-1] for crate in part2_state ])

'TZLTLWRNF'